In [ ]:
#initial stetup

import sys
sys.path.append(r'Desktop\SMU_files\\')
import Keithley2401_voltmeter_060823 as K2401
import pyvisa
import time
import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

rm = pyvisa.ResourceManager()
SMU_RM = rm.open_resource('GPIB0::3::INSTR')
SMU = K2401.Keithley2401(SMU_RM)
SMU.initial_setup()

In [ ]:
plt.ion()

class DynamicUpdate():
    
    def on_launch(self, x_label, y_label):
        #set up plot
        self.figure, self.ax = plt.subplots()
        self.lines, = self.ax.plot([],[], 'o')
        #set up axis which scale with data
        self.ax.set_autoscalex_on(True)
        self.ax.set_autoscaley_on(True)
        #set up figure labels
        self.ax.set_xlabel(x_label)
        self.ax.set_ylabel(y_label)
        self.ax.grid
        ...

    def on_running(self, xdata, ydata):
        #update data (with the new _and_ the old points)
        self.lines.set_xdata(xdata)
        self.lines.set_ydata(ydata)
        #need both of these in order to rescale
        self.ax.relim()
        self.ax.autoscale_view()
        #we need to draw *and* flush
        self.figure.canvas.draw()
        self.figure.canvas.flush_events()
        
    def on_completion(self, fname, level):
        #saves data to a .png file
        from datetime import date
        d = date.today()
        self.figure.tight_layout
        self.figure.savefig(F'{d.strftime("%Y_%m_%d")}_{fname}_{level}.png', dpi = 300, bbox_inches = 'tight')
        

In [ ]:
#constant current, read voltage

time0 = time.time()
current_level = 10e-3 # set a constant current
measurement_time = 30 # number of seconds to perform measurement
num_readings = 60 # number of data points to collect
wait_time = measurement_time/num_readings # wait time between measurements in seconds

NPLC = 1
I_range = 10e-3 # current range
V_compliance = 5 # max voltage
init_wait = 0.25 # does not do anything yet
SMU.setup_single_Vmeas(NPLC = NPLC, I_range = I_range, V_compliance = V_compliance, current_level = current_level, init_wait = init_wait)
SMU.turn_on()

#initializes dynamically updating plots
cv = DynamicUpdate()
tv = DynamicUpdate()

cv.on_launch("current (mA)", "voltage (V)")
tv.on_launch("time (s)", "voltage (V)")

#creates lists to store data
currents = []
times = []
voltages = []

#generates data and accounts for time lag
for i in range(1,num_readings):
    time_1 = time.time()
    
    I,V = SMU.single_Vmeas()
    currents.append(I)
    voltages.append(V)
    times.append(time.time()-time0)
    cv.on_running(currents, voltages)
    tv.on_running(times, voltages)
    
    time_2 = time.time()
    time_error = time_2 - time_1
    
    if time_error < wait_time:
        time.sleep(wait_time - time_error)
    else:
        time.sleep(time_error)

SMU.turn_off()

#stores data and prints a figure displaying voltage vs. time
d = datetime.date.today()
np.savetxt(F'{d.strftime("%Y_%m_%d")}_CV_data_{current_level}amp.csv', np.c_[np.asarray(currents), np.asarray(voltages)], delimiter = ',')
np.savetxt(F'{d.strftime("%Y_%m_%d")}_TV_data.csv', np.c_[np.asarray(times), np.asarray(voltages)], delimiter = ',')
tv.on_completion("CC", F'{current_level}amp')

In [ ]:
time0 = time.time()
voltage_level = 3 # set a constant voltage
measurement_time = 10 # number of seconds to perform measurement
num_readings = 20 # number of data points to collect
wait_time = measurement_time/num_readings # wait time between measurements in seconds

NPLC = 1
V_range = 20 # voltage range
I_compliance = 1e-1 # max current
V_compliance = 5 # max voltage
init_wait = 0.25 # does not do anything yet
SMU.setup_single_Imeas(NPLC = NPLC, V_range = V_range, V_compliance = V_compliance, I_compliance = I_compliance, voltage_level = voltage_level, init_wait = init_wait)
SMU.turn_on()

#initializes dynamically updating plots
vc = DynamicUpdate()
tc = DynamicUpdate()

vc.on_launch("voltage (V)", "current (mA)")
tc.on_launch("time (s)", "current (mA)")

#creates lists to store data
currents = []
times = []
voltages = []

#generates data and accounts for time lag
for i in range(1,num_readings):
    time_1 = time.time()
    
    I,V = SMU.single_Vmeas()
    currents.append(I)
    voltages.append(V)
    times.append(time.time()-time0)
    vc.on_running(voltages, currents)
    tc.on_running(times, currents)
    
    time_2 = time.time()
    time_error = time_2 - time_1
    
    if time_error < wait_time:
        time.sleep(wait_time - time_error)
    else:
        time.sleep(time_error)

SMU.turn_off()

#stores data and prints a figure displaying current vs. time
d = datetime.date.today()
np.savetxt(F'{d.strftime("%Y_%m_%d")}_VC_data_{voltage_level}volt.csv', np.c_[np.asarray(voltages), np.asarray(currents)], delimiter = ',')
np.savetxt(F'{d.strftime("%Y_%m_%d")}_TC_data.csv', np.c_[np.asarray(times), np.asarray(currents)], delimiter = ',')
tc.on_completion("CP", F'{voltage_level}volt')